In [1]:
from datetime import datetime
import requests
import numpy as np
import pandas as pd
import yaml
import re
from bs4 import BeautifulSoup
import wikitextparser as wtp
from ratelimit import rate_limited
from operator import *

def save_obj(obj, name ):
    import pickle
    with open('obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
        
def load_obj(name ):
    import pickle
    with open('obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

def easyText(t):
    return re.sub(r'\W+', '', str(t)).lower()

In [4]:
r = requests.get(url='http://liquipedia.net/dota2/api.php?action=ask&format=json&query=%5B%5BCategory%3ATeams%5D%5D%7C%3FHas+id%7C%3FHas+id+lc%7C%3FHas+location%7Csort%3DHas+id%7Climit%3D1000')
json2 = r.json()

In [5]:
df = pd.DataFrame(columns=['name', 'id', 'id lc', 'location'])
idlcs = []

In [6]:
for lname, ldata in json2.get('query').get('results').items():
    po = ldata.get('printouts')
    pid = ldata.get('fulltext')
    try:
        idn = po.get('Has id')[0]
    except IndexError:
        idn = 'Nan' 
    try:
        idlc = po.get('Has id lc')[0]
    except IndexError:
        idlc = 'Nan'
    try:
        loc = po.get('Has location')[0]
    except IndexError:
        loc = 'Nan'        
    df = df.append({'name': lname, 'id': idn, 'id lc': idlc, 'location': loc}, ignore_index=True)
    idlcs.append(idlc)

In [15]:
slash = []
for t in idlcs:
    if '/' in t:
        slash.append(re.search(r'(.+)/(.+)', t)[1])
        slash.append(re.search(r'(.+)/(.+)', t)[2])
slash = list(set(slash))

In [25]:
ab = idlcs + slash

In [18]:
@rate_limited(1, 2)
def requestQuery(title):
    headers = {
    'User-Agent': 'Data for Research',
    'From': 'terthasarit@live.com', 
    'Accept-Encoding': 'gzip'
    }
    r = requests.get(url='http://liquipedia.net/dota2/api.php?action=query&format=json&prop=linkshere&lhnamespace=10&lhlimit=500&titles=Template%3ATeam%2F' + title, headers=headers)
    json = r.json()
    return json

In [19]:
AlternateIdDict = {}
json = requestQuery('Nan')
for pageid, ldata in json.get('query').get('pages').items():
    mainid = easyText(ldata['title'])
    try:
        for altid in ldata['linkshere']:
            AlternateIdDict[re.search(r'Template:Team/(.+)', altid['title'])[1]] = 'Nan'
    except KeyError:
        AlternateIdDict['Nan'] = 'Nan'

In [20]:
AlternateIdDict

{'Nan': 'Nan'}

In [39]:
teamShort = {}
teamEasy = {}

In [40]:
%%time
for title in ab:
    json = requestQuery(title)
    for pageid, ldata in json.get('query').get('pages').items():
        mainid = easyText(ldata['title'])
        try:
            for altid in ldata['linkshere']:
                try:
                    teamShort[re.search(r'Template:Team/(.+)', altid['title'])[1]] = title
                    teamEasy[re.search(r'Template:Team/(.+)', altid['title'])[1]] = easyText(title)                    
                except TypeError:
                    print(title + ' NoneType')
                    continue
        except KeyError:
            teamShort[title] = title
            teamEasy[easyText(title)] = easyText(title)


team liquid NoneType
team liquid NoneType
Wall time: 21min 11s


In [41]:
teamShort

{'.aspera.': 'aspera',
 '1stvn': '1st.vn',
 '3dclan': '3d!clan',
 '3dmax': '3dmax',
 '4 anchors + sea captain': '4 anchors + sea captain',
 '4 clover & lepricon': '4 clover & lepricon',
 '4 friends + chrillee': '4 friends + chrillee',
 '4 vikings + dane': '4 vikings + dane',
 '4fc': '4 friends + chrillee',
 '4p5': '4 protect five',
 '5eva.': '5eva',
 '5inq': '5 in qo',
 '5jungz': '5jungz',
 '90s': '90s gaming',
 '???': 're-defining madness',
 'ASUS.polar': 'asus polar',
 'Fiskestanga': 'fiskestanga',
 'Flip.sid3 Tactics': 'flipsid3 tactics',
 'MUFC': 'mufc',
 'Nan': 'Nan',
 'ROOT Gaming': 'root gaming',
 'Team Finland': 'finland',
 'aaa': 'team aaa',
 'abc': 'apple banana cucumber',
 'absolutelegends': 'absolute legends',
 'aces': 'aces gaming',
 'acion arena': 'acion arena',
 'ad finem': 'ad finem',
 'affection': 'affection!',
 'aftershock': 'aftershock gaming',
 'ahead': 'ahead gaming',
 'ahead.kz': 'ahead gaming',
 'al': 'absolute legends',
 'alternate': 'alternate attax',
 'ancient

In [42]:
teamEasy

{'.aspera.': 'aspera',
 '1stvn': '1stvn',
 '3dclan': '3dclan',
 '3dmax': '3dmax',
 '4 friends + chrillee': '4friendschrillee',
 '4anchorsseacaptain': '4anchorsseacaptain',
 '4cloverlepricon': '4cloverlepricon',
 '4fc': '4friendschrillee',
 '4p5': '4protectfive',
 '4vikingsdane': '4vikingsdane',
 '5eva.': '5eva',
 '5inq': '5inqo',
 '5jungz': '5jungz',
 '90s': '90sgaming',
 '???': 'redefiningmadness',
 'ASUS.polar': 'asuspolar',
 'Fiskestanga': 'fiskestanga',
 'Flip.sid3 Tactics': 'flipsid3tactics',
 'MUFC': 'mufc',
 'ROOT Gaming': 'rootgaming',
 'Team Finland': 'finland',
 'aaa': 'teamaaa',
 'abc': 'applebananacucumber',
 'absolutelegends': 'absolutelegends',
 'aces': 'acesgaming',
 'acionarena': 'acionarena',
 'adfinem': 'adfinem',
 'affection': 'affection',
 'aftershock': 'aftershockgaming',
 'ahead': 'aheadgaming',
 'ahead.kz': 'aheadgaming',
 'al': 'absolutelegends',
 'alternate': 'alternateattax',
 'ancientwarriors': 'ancientwarriors',
 'animal kingdom': 'kingdomdota',
 'anji': 'cy

In [43]:
save_obj(teamShort, 'dict/teamShort2')
save_obj(teamEasy, 'dict/teamEasy2')

In [104]:
#save_obj(teamShort, 'dict/teamShort')

In [ ]:
df['idlow'] = df.id

In [ ]:
df.idlow = df.idlow.str.replace(r'\W+', '').str.lower()

In [ ]:
df[df['player'] == 'kpii']

In [ ]:
df.to_csv('idanddob.csv', index=False)

In [111]:
teamShort['dk']

'team dk'

In [112]:
json

{'batchcomplete': '',
 'query': {'pages': {'2395': {'linkshere': [{'ns': 10,
      'pageid': 2396,
      'redirect': '',
      'title': 'Template:Team/dk'},
     {'ns': 10,
      'pageid': 46532,
      'redirect': '',
      'title': 'Template:Team/team dk/team dk'}],
    'ns': 10,
    'pageid': 2395,
    'title': 'Template:Team/team dk'}}}}